In [1]:
from netCDF4 import Dataset
import numpy as np 
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy.ma as ma
from cartopy.io.shapereader import Reader
from cartopy.feature import ShapelyFeature
import pyart
import matplotlib.lines as mlines
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
import matplotlib.font_manager as fm
import pickle
from datetime import datetime, timedelta
from metpy.interpolate.grid import natural_neighbor_to_grid
from metpy.interpolate import interpolate_to_points
from metpy.interpolate import interpolate_to_grid
import csv


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



C:\Users\matts\anaconda3\envs\spork2022\lib\site-packages\xarray\core\pycompat.py:22: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  duck_array_version = LooseVersion(duck_array_module.__version__)
C:\Users\matts\anaconda3\envs\spork2022\lib\site-packages\xarray\core\pycompat.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  duck_array_version = LooseVersion("0.0.0")
C:\Users\matts\anaconda3\envs\spork2022\lib\site-packages\xarray\core\pycompat.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  duck_array_version = LooseVersion("0.0.0")
C:\Users\matts\anaconda3\envs\spork2022\lib\site-packages\setuptools\_distutils\version.py:346: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
C:\Users\matts\anaconda3\envs\spork2022\lib\site-packages\setuptools\_distutils\version

In [2]:
# MIT License

# Copyright (c) 2017 thunderhoser

# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:

# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
# SOFTWARE.

import numpy
from scipy.interpolate import interp1d


def _run_pmm_one_variable(
        input_matrix, max_percentile_level=100):
    """Applies PMM to one variable.
    E = number of examples (realizations over which to average)
    :param input_matrix: numpy array.  The first axis must have length E.  Other
        axes are assumed to be spatial dimensions.  Thus, input_matrix[i, ...]
        is the spatial field for the [i]th example.
    :param max_percentile_level: Maximum percentile.  No output value will
        exceed the [q]th percentile of `input_matrix`, where q =
        `max_percentile_level`.  Similarly, no output value will be less than
        the [100 - q]th percentile of `input_matrix`.
    :return: mean_field_matrix: numpy array of probability-matched means.  Will
        have the same dimensions as `input_matrix`, except without the first
        axis.  For example, if `input_matrix` is E x 32 x 32 x 12, this will be
        32 x 32 x 12.
    """

    # Pool values over all dimensions and remove extremes.
    pooled_values = numpy.ravel(input_matrix)
    pooled_values = numpy.sort(pooled_values)

    max_pooled_value = numpy.percentile(pooled_values, max_percentile_level)
    pooled_values = pooled_values[pooled_values <= max_pooled_value]

    min_pooled_value = numpy.percentile(
        pooled_values, 100 - max_percentile_level)
    pooled_values = pooled_values[pooled_values >= min_pooled_value]

    # Find ensemble mean at each grid point.
    mean_field_matrix = numpy.mean(input_matrix, axis=0)
    mean_field_flattened = numpy.ravel(mean_field_matrix)

    # At each grid point, replace ensemble mean with the same percentile from
    # pooled array.
    pooled_value_percentiles = numpy.linspace(
        0, 100, num=len(pooled_values), dtype=float)
    mean_value_percentiles = numpy.linspace(
        0, 100, num=len(mean_field_flattened), dtype=float)

    sort_indices = numpy.argsort(mean_field_flattened)
    unsort_indices = numpy.argsort(sort_indices)

    interp_object = interp1d(
        pooled_value_percentiles, pooled_values, kind='linear',
        bounds_error=True, assume_sorted=True)

    mean_field_flattened = interp_object(mean_value_percentiles)
    mean_field_flattened = mean_field_flattened[unsort_indices]
    mean_field_matrix = numpy.reshape(
        mean_field_flattened, mean_field_matrix.shape)

    return mean_field_matrix

In [3]:
#Rotation code from https://stackoverflow.com/questions/29708840/rotate-meshgrid-with-numpy
def DoRotation(xspan, yspan, RotRad=0):
    """Generate a meshgrid and rotate it by RotRad radians."""

    # Clockwise, 2D rotation matrix
    RotMatrix = np.array([[np.cos(RotRad),  np.sin(RotRad)],
                          [-np.sin(RotRad), np.cos(RotRad)]])

    x, y = np.meshgrid(xspan, yspan)
    return np.einsum('ji, mni -> jmn', RotMatrix, np.dstack([x, y]))

In [4]:
#leg1_PDD = Dataset('D:/P3_Syntheses/PDDkd_synthesis_2019_06_08_204430.nc')

In [5]:
#Read in the sounding data from the spreadsheet
#Bring in the sounding data
CIsoundings=np.genfromtxt('SupercellSoundings/environment_cicases_Final.csv',skip_header=1,delimiter=',', dtype=str)
Oldsoundings=np.genfromtxt('SupercellSoundings/environment_Oldcases.csv',delimiter=',', dtype=str)
# ETsoundings=np.genfromtxt('PythonEnvVars1.csv', skip_header=1,delimiter=',')
# ETsoundings[ETsoundings==-9999]=np.nan

NewSoundings=np.genfromtxt('SupercellSoundingsNew/environment_cicases1022.csv', delimiter=',', dtype=str)

TORsondes = pickle.load(open('TORsondes.pkl', 'rb'))
NTORsondes = pickle.load(open('NTORsondes.pkl', 'rb'))
ETORsondes = pickle.load(open('ETORsondes.pkl', 'rb'))
ENTORsondes = pickle.load(open('ENTORsondes.pkl', 'rb'))
TTorsondes = pickle.load(open('TTorsondes.pkl', 'rb'))
TNTsondes = pickle.load(open('TNTsondes.pkl', 'rb'))
STsondes = pickle.load(open('STsondes.pkl', 'rb'))
SNsondes = pickle.load(open('SNsondes.pkl', 'rb'))

ALL_sites = pickle.load(open('ALLVARSite.pkl', 'rb'))
ALL_years = pickle.load(open('ALLVARYear.pkl', 'rb'))
ALL_months = pickle.load(open('ALLVARMonth.pkl', 'rb'))
ALL_days = pickle.load(open('ALLVARDay.pkl', 'rb'))
ALL_hours = pickle.load(open('ALLVARHour.pkl', 'rb'))
ALL_TN = pickle.load(open('ALLVARTor_Non.pkl', 'rb'))

ALL_b_t = np.zeros((ALL_TN.shape))
ALL_b_t[ALL_TN=='T'] = 1.0

#Fix the incorrect entry here
ALL_sites[44] = 'KTLX'
ALL_years[44] = 2019
ALL_months[44] = 3
ALL_days[44] = 23
ALL_hours[44] = 23
ALL_TN[44] = 'N'

#Fix second incorrect entry here
ALL_sites[54] = 'KFDR'
ALL_years[54] = 2013
ALL_months[54] = 4
ALL_days[54] = 18
ALL_hours[54] = 1
ALL_TN[54] = 'T'

#Fix third incorrect entry here
ALL_sites[61] = 'KENX'
ALL_years[61] = 2014
ALL_months[61] = 7
ALL_days[61] = 3
ALL_hours[61] = 21
ALL_TN[61] = 'N'

#Fix fourth incorrect entry here
ALL_sites[137] = 'KLZK'
ALL_years[137] = 2014
ALL_months[137] = 4
ALL_days[137] = 28
ALL_hours[137] = 1
ALL_TN[137] = 'T'

obsd_TOR = pickle.load(open('sp6_dir_TOR.pkl', 'rb'))
obsd_NTOR = pickle.load(open('sp6_dir_NTOR.pkl', 'rb'))
obsd_ETOR = pickle.load(open('sp6_dir_ETOR.pkl', 'rb'))
obsd_ENTOR = pickle.load(open('sp6_dir_ENTOR.pkl', 'rb'))
obsd_TTor = pickle.load(open('sp6_dir_TTor.pkl', 'rb'))
obsd_TNT = pickle.load(open('sp6_dir_TNT.pkl', 'rb'))
obsd_ST = pickle.load(open('sp6_dir_ST.pkl', 'rb'))
obsd_SN = pickle.load(open('sp6_dir_SN.pkl', 'rb'))

obspd_TOR = pickle.load(open('sp6_sp_TOR.pkl', 'rb'))
obspd_NTOR = pickle.load(open('sp6_sp_NTOR.pkl', 'rb'))
obspd_ETOR = pickle.load(open('sp6_sp_ETOR.pkl', 'rb'))
obspd_ENTOR = pickle.load(open('sp6_sp_ENTOR.pkl', 'rb'))
obspd_TTor = pickle.load(open('sp6_sp_TTor.pkl', 'rb'))
obspd_TNT = pickle.load(open('sp6_sp_TNT.pkl', 'rb'))
obspd_ST = pickle.load(open('sp6_sp_ST.pkl', 'rb'))
obspd_SN = pickle.load(open('sp6_sp_SN.pkl', 'rb'))

#Create the full arrays for storm speed and direction
all_dir = np.concatenate([obsd_TOR, obsd_NTOR, obsd_ETOR, obsd_ENTOR, obsd_TTor, obsd_TNT, obsd_ST, obsd_SN], axis=0)
all_spd = np.concatenate([obspd_TOR, obspd_NTOR, obspd_ETOR, obspd_ENTOR, obspd_TTor, obspd_TNT, obspd_ST, obspd_SN], axis=0)

#Get FFD angle dataset
ffdA = 190
ffdBB = 190
ffdCC = 190
ffdEE = 200
ffdFF = 230
ffdH = 150
ffdHH = 210
ffdI = 160
ffdJ = 170
ffdK = 120
ffdR = 200
ffdS = 160
ffdGG = 180
ffdE = 230

ffd1 = 140
ffd3 = 220
ffd6 = 190
ffd15 = 170
ffd22 = 130
ffd10 = 170
ffd13 = 150
ffd14 = 160
ffd16 = 180
ffd19 = 200
ffd21 = 150
ffd23 = 180
ffd24 = 190
ffd26 = 200
ffd27 = 160
ffd28 = 180
ffd29 = 180
ffd30 = 210
ffd32 = 160

ffdet1 = 180
ffdet3 = 165
ffdet6 = 125
ffdet8 = 170
ffdet9 = 205
ffdet11 = 190
ffdet13 = 180
ffdet14 = 220
ffdet16 = 145
ffdet20 = 190
ffdet23 = 195
ffdet26 = 170
ffdet33 = 110
ffdet34 = 150
ffdet35 = 180
ffdet36 = 140

ffden1 = 130
ffden2 = 165
ffden4 = 140
ffden7 = 180
ffden8 = 180
ffden9 = 150
ffden10 = 170
ffden11 = 125
ffden12 = 170

ffdcit1 = 225
ffdcit2 = 180
ffdcit3 = 190
ffdcit4 = 225
ffdcit5 = 155
ffdcit6 = 190
ffdcit8 = 200
ffdcit9 = 170
ffdcit10 = 190
ffdcit11 = 165
ffdcit12 = 190
ffdcit13 = 150
ffdcit14 = 180
ffdcit15 = 140
ffdcit17 = 190
ffdcit18 = 260
ffdcit19 = 190
ffdcit20 = 165
ffdcit22 = 190
ffdcit23 = 185
ffdcit24 = 130
ffdcit26 = 120
ffdcit27 = 130
ffdcit28 = 165
ffdcit29 = 170
ffdcit30 = 155
ffdcit31 = 145
ffdcit32 = 200
ffdcit34 = 170

ffdcin1 = 190
ffdcin2 = 180
ffdcin3 = 190
ffdcin4 = 180
ffdcin5 = 150
ffdcin6 = 210
ffdcin9 = 200
ffdcin10 = 160
ffdcin11 = 185
ffdcin12 = 140
ffdcin13 = 170
ffdcin14 = 195
ffdcin15 = 185
ffdcin16 = 150
ffdcin17 = 165
ffdcin18 = 165

ffdSN1 = 195
ffdSN2 = 160
ffdSN3 = 160 
ffdSN4 = 175
ffdSN5 = 170
ffdSN6 = 180
ffdSN7 = 145
ffdSN8 = 205
ffdSN9 = 200
ffdSN10 = 220
ffdSN11 = 180
ffdSN12 = 175
ffdSN13 = 135
ffdSN14 = 200
ffdSN15 = 235
ffdSN16 = 180
ffdSN17 = 180
ffdSN18 = 175
ffdSN19 = 225
ffdSN20 = 155
ffdSN21 = 150
ffdSN22 = 180
ffdSN23 = 185
ffdSN24 = 180
ffdSN25 = 190
ffdSN26 = 160
ffdSN27 = 160
ffdSN28 = 240
ffdSN29 = 180
ffdSN30 = 245
ffdSN31 = 125
ffdSN32 = 130
ffdSN33 = 210
ffdSN34 = 175
ffdSN35 = 165
ffdSN36 = 140
ffdSN37 = 170
ffdSN38 = 175
ffdSN39 = 180
ffdSN40 = 210
ffdSN42 = 195
ffdSN43 = 170
ffdSN44 = 180
ffdSN45 = 210
ffdSN46 = 225
ffdSN47 = 150
ffdSN48 = 230
ffdSN49 = 200
ffdSN50 = 190

ffdSN51 = 190
ffdSN52 = 220
ffdSN53 = 220
ffdSN54 = 215
ffdSN55 = 170

ffdSN56 = 195
ffdSN57 = 180
ffdSN58 = 155
ffdSN59 = 185
ffdSN60 = 190


ffdST1 = 115
ffdST2 = 150
ffdST3 = 150
ffdST4 = 175
ffdST5 = 200
ffdST6 = 115
ffdST7 = 160
ffdST8 = 120
ffdST9 = 180
ffdST10 = 155
ffdST11 = 160
ffdST12 = 180
ffdST13 = 225
ffdST14 = 210
ffdST15 = 180
ffdST16 = 155
ffdST17 = 180
ffdST18 = 185
ffdST19 = 170
ffdST20 = 135
ffdST21 = 185
ffdST24 = 160
ffdST27 = 180
ffdST28 = 165
ffdST30 = 245
ffdST31 = 155
ffdST33 = 180
ffdST34 = 140
ffdST35 = 140
ffdST37 = 185
ffdST38 = 210
ffdST39 = 155
ffdST40 = 205
ffdST41 = 140
ffdST42 = 150
ffdST43 = 160
ffdST44 = 200
ffdST45 = 185
ffdST47 = 195
ffdST48 = 190
ffdST50 = 185
ffdST51 = 170
ffdST52 = 205
ffdST53 = 240

ffds = np.asarray([ffdcit1, ffdcit2, ffdcit3, ffdcit4, ffdcit5, ffdcit6, ffdcit8, ffdcit9, ffdcit10,
        ffdcit11, ffdcit12, ffdcit13, ffdcit14, ffdcit15, ffdcit17, ffdcit18, ffdcit19, ffdcit20, ffdcit22, ffdcit23,
        ffdcit24, ffdcit26, ffdcit27, ffdcit28, ffdcit29, ffdcit30, ffdcit31, ffdcit32, ffdcit34, ffdcin1, ffdcin2,
        ffdcin3, ffdcin4, ffdcin5, ffdcin6, ffdcin9, ffdcin10, ffdcin11, ffdcin12, ffdcin13, ffdcin14, ffdcin15, ffdcin16,
        ffdcin17, ffdcin18,ffdet1, ffdet3, ffdet6, ffdet8, ffdet9, ffdet11, ffdet13, ffdet14, ffdet16, ffdet20,
        ffdet23, ffdet26, ffdet33, ffdet34, ffdet35, ffdet36, ffden1, ffden2, ffden4, ffden7, ffden8, ffden9, 
        ffden10, ffden11, ffden12,ffdA, ffdBB, ffdCC, ffdEE, ffdFF, ffdH, ffdHH, ffdI, ffdJ, ffdK, ffdR, ffdS, ffdGG, ffdE, ffd1, ffd3, ffd6,
        ffd15, ffd22, ffd10, ffd13, ffd14, ffd16, ffd19, ffd21, ffd23, ffd24, ffd26, ffd27, ffd28, ffd29, 
        ffd30, ffd32,ffdST1, ffdST2, ffdST3, ffdST4, ffdST5, ffdST6, ffdST7, ffdST8, ffdST9, ffdST10, ffdST11,
        ffdST12, ffdST13, ffdST14, ffdST15, ffdST16, ffdST17, ffdST18, ffdST19, ffdST20, ffdST21, ffdST24, ffdST27, 
        ffdST28, ffdST30, ffdST31, ffdST33, ffdST34, ffdST35, ffdST37, ffdST38, ffdST39, ffdST40, ffdST41, ffdST42,
        ffdST43, ffdST44, ffdST45, ffdST47, ffdST48, ffdST50, ffdST51, ffdST52, ffdST53,ffdSN1,
        ffdSN2,ffdSN3,ffdSN4,ffdSN5,ffdSN6,ffdSN7,ffdSN8,ffdSN9,ffdSN10,ffdSN11,ffdSN12,ffdSN13,ffdSN14,ffdSN15,ffdSN16,ffdSN17,
        ffdSN18,ffdSN19,ffdSN20,ffdSN21,ffdSN22,ffdSN23,ffdSN24,ffdSN25,ffdSN26,ffdSN27,ffdSN28,ffdSN29,ffdSN30,ffdSN31,ffdSN32,ffdSN33,
        ffdSN34,ffdSN35,ffdSN36,ffdSN37,ffdSN38,ffdSN39,ffdSN40,ffdSN42,ffdSN43,ffdSN44,ffdSN45,ffdSN46,ffdSN47,ffdSN48,ffdSN49,ffdSN50,
        ffdSN51,ffdSN52,ffdSN53,ffdSN54,ffdSN55,ffdSN56,ffdSN57,ffdSN58,ffdSN59,ffdSN60])

In [6]:
def rotate_stamp(ffd, ncfile, xoff, yoff, st_2a, st_2b):
    ref = ncfile.variables['REFL'][:]
    kdp = ncfile.variables['KDP'][:]
    zdr = ncfile.variables['ZDR'][:]
    cc = ncfile.variables['CC'][:]
    zdr[ref < 20] = np.nan
    zdr[cc < 0.9] = np.nan
    kdp[ref < 20] = np.nan
    kdp[cc < 0.9] = np.nan
    nzdr = ncfile.variables['NZDR'][:]
    nzdr[ref[:,4,:,:] < 20] = 0.0
    nzdr[cc[:,4,:,:] < 0.95] = 0.0
    zdrd = ncfile.variables['ZDRD'][:]
    rot = ncfile.variables['ROT'][:]
    times = ncfile.variables['Times'][:]
    azim = ncfile.variables['azim'][:]
    vel = ncfile.variables['VEL'][:]
    
    try:
        ncfile7 = Dataset('D:/NewSPORK_stamps/SPORK_RERUN'+str(ALL_years[i])+str(ALL_months[i])+str(int(ALL_days[i]))+str(ALL_sites[i])+'.nc')
        times = ncfile7.variables['Times'][:]

    except:
        ncfile7 = Dataset('D:/NewSPORK_stamps/SPORK_RERUN'+str(ALL_years[i])+str(ALL_months[i])+str(int(ALL_days[i])-1)+str(ALL_sites[i])+'.nc')
        times = ncfile7.variables['Times'][:]

    st_1 = datetime(int(ALL_years[i]), int(ALL_months[i]), int(ALL_days[i]), int(ALL_hours[i]), 0)-timedelta(minutes=30)
    print(st_1)
    time_diffs = []
    for j in range(len(times)):

        time_diff = (datetime.fromtimestamp(times[j]) - st_1).total_seconds()
        time_diffs.append(time_diff)
    time_array = np.asarray(time_diffs)  
    time_array[time_array<0] = 9999
    
    #Set up logic to clip the time array to remove times when the storm is
    #too far from the radar. Set the time_diffs at those times to be 9999.
    #Removing times before the start time
    if st_2a:
        time_diffs2 = []
        for j in range(len(times)):
            time_diff2 = (datetime.fromtimestamp(times[j]) - st_2a).total_seconds()
            time_diffs2.append(time_diff2)
        time_array2 = np.asarray(time_diffs2)  
        time_array[time_array2<0] = 9999
        
    #Removing times after the end time
    if st_2b:
        time_diffs3 = []
        for j in range(len(times)):
            time_diff3 = (datetime.fromtimestamp(times[j]) - st_2b).total_seconds()
            time_diffs3.append(time_diff3)
        time_array3 = np.asarray(time_diffs3)  
        time_array[time_array3>0] = 9999
    
    print(time_array)
    print(time_diffs)
    print(np.where(time_array < 3600))
    print(ref[time_array < 3600, :, :, :].shape[0], 'times')

    ref_mn = ref[time_array < 3600, :, :, :]
    zdr_mn = zdr[time_array < 3600, :, :, :]
    kdp_mn = kdp[time_array < 3600, :, :, :]
    cc_mn = cc[time_array < 3600, :, :, :]
    rot_mn = rot[time_array < 3600, :, :, :]
    zdrd_mn = zdrd[time_array < 3600, :, :]
    nzdr_mn = nzdr[time_array < 3600, :, :]

    ref_st = np.nanmean(ref_mn, axis=0)
    zdr_st = np.nanmean(zdr_mn, axis=0)
    kdp_st = np.nanmean(kdp_mn, axis=0)
    cc_st = np.nanmean(cc_mn, axis=0)
    rot_st = np.nanmean(rot_mn, axis=0)
    zdrd_st = np.nanmean(zdrd_mn, axis=0)
    nzdr_st = np.nanmean(nzdr_mn, axis=0)

    nzdr_mn[np.isnan(nzdr_mn)] = 0.0
    zdrd_mn[np.isnan(zdrd_mn)] = 0.0
    zdr_mn[np.isnan(zdr_mn)] = 0.0
    kdp_mn[np.isnan(kdp_mn)] = 0.0
    cc_mn[np.isnan(cc_mn)] = 0.0
    rot_mn[ref_mn < 20.0] = 0.0

    ref_pm = _run_pmm_one_variable(ref_mn)
    rot_pm = _run_pmm_one_variable(rot_mn)
    zdrd_pm =  _run_pmm_one_variable(zdrd_mn)
    nzdr_pm =  _run_pmm_one_variable(nzdr_mn)
    zdr_pm =  _run_pmm_one_variable(zdr_mn)
    kdp_pm =  _run_pmm_one_variable(kdp_mn)
    cc_pm =  _run_pmm_one_variable(cc_mn)
    for q in range(zdr_mn.shape[0]):
        print(np.max(rot_mn[q,12,:,:]))
    print(ref_mn.shape)
    xr = np.arange(-70,70,1)
    xra1, yra1 = DoRotation(xr, xr, RotRad=np.deg2rad((ffd)-180)*-1)
    xri, yri, Z_interpi = interpolate_to_grid(np.ndarray.flatten(xra1), np.ndarray.flatten(yra1),
                                             np.ndarray.flatten(ref_pm[4,:,:]), hres=1,
                                             boundary_coords={'west':-50+xoff,'south':-50+yoff,'east':49+xoff,'north':49+yoff})
    xri, yri, rot1_interpi = interpolate_to_grid(np.ndarray.flatten(xra1), np.ndarray.flatten(yra1),
                                             np.ndarray.flatten(rot_pm[4,:,:]), hres=1,
                                             boundary_coords={'west':-50+xoff,'south':-50+yoff,'east':49+xoff,'north':49+yoff})
    xri, yri, rot3_interpi = interpolate_to_grid(np.ndarray.flatten(xra1), np.ndarray.flatten(yra1),
                                             np.ndarray.flatten(rot_pm[12,:,:]), hres=1,
                                             boundary_coords={'west':-50+xoff,'south':-50+yoff,'east':49+xoff,'north':49+yoff})
    xri, yri, rot5_interpi = interpolate_to_grid(np.ndarray.flatten(xra1), np.ndarray.flatten(yra1),
                                             np.ndarray.flatten(rot_pm[20,:,:]), hres=1,
                                             boundary_coords={'west':-50+xoff,'south':-50+yoff,'east':49+xoff,'north':49+yoff})
    xri, yri, rot7_interpi = interpolate_to_grid(np.ndarray.flatten(xra1), np.ndarray.flatten(yra1),
                                             np.ndarray.flatten(rot_pm[28,:,:]), hres=1,
                                             boundary_coords={'west':-50+xoff,'south':-50+yoff,'east':49+xoff,'north':49+yoff})
    xri, yri, zdrd_interpi = interpolate_to_grid(np.ndarray.flatten(xra1), np.ndarray.flatten(yra1),
                                             np.ndarray.flatten(zdrd_pm[:,:]), hres=1,
                                             boundary_coords={'west':-50+xoff,'south':-50+yoff,'east':49+xoff,'north':49+yoff})
    xri, yri, zdr_interpi = interpolate_to_grid(np.ndarray.flatten(xra1), np.ndarray.flatten(yra1),
                                             np.ndarray.flatten(zdr_pm[4,:,:]), hres=1,
                                             boundary_coords={'west':-50+xoff,'south':-50+yoff,'east':49+xoff,'north':49+yoff})
    xri, yri, nzdr_interpi = interpolate_to_grid(np.ndarray.flatten(xra1), np.ndarray.flatten(yra1),
                                             np.ndarray.flatten(nzdr_pm[:,:]), hres=1,
                                             boundary_coords={'west':-50+xoff,'south':-50+yoff,'east':49+xoff,'north':49+yoff})
    xri, yri, kdp_interpi = interpolate_to_grid(np.ndarray.flatten(xra1), np.ndarray.flatten(yra1),
                                             np.ndarray.flatten(kdp_pm[4,:,:]), hres=1,
                                             boundary_coords={'west':-50+xoff,'south':-50+yoff,'east':49+xoff,'north':49+yoff})
    xri, yri, cc_interpi = interpolate_to_grid(np.ndarray.flatten(xra1), np.ndarray.flatten(yra1),
                                             np.ndarray.flatten(cc_pm[4,:,:]), hres=1,
                                             boundary_coords={'west':-50+xoff,'south':-50+yoff,'east':49+xoff,'north':49+yoff})
    return xri, yri, Z_interpi, rot1_interpi, rot3_interpi, rot5_interpi, rot7_interpi, zdrd_interpi, nzdr_interpi, zdr_interpi, kdp_interpi, cc_interpi

In [7]:
#Get the list of meso locations
meso_locs = np.genfromtxt('MesocycloneLocationsPass3.csv', delimiter=',', skip_header=1)
meso_x = meso_locs[:,3]
meso_y = meso_locs[:,4]

In [8]:
#print(meso_x)

In [39]:
#Generate the file names for the postage stamps
j=1
#for i in range(len(ALL_years)):
#for i in np.arange(126,205,1):
#for i in np.arange(1,206,1):

for i in [203]:
    try:
        print('D:/SPORK_AC/SPORK_AC'+str(ALL_years[i])+str(ALL_months[i])+str(ALL_days[i])+str(ALL_sites[i])+'.nc')
        ncfile = Dataset('D:/SPORK_AC/SPORK_AC'+str(ALL_years[i])+str(ALL_months[i])+str(ALL_days[i])+str(ALL_sites[i])+'.nc')
    except:
        try:
            ncfile = Dataset('D:/SPORK_AC/SPORK_AC'+str(ALL_years[i])+str(ALL_months[i])+str(int(ALL_days[i])-1)+str(ALL_sites[i])+'.nc')
        except:
            print('SPORK_RERUN'+str(ALL_years[i])+str(ALL_months[i])+str(ALL_days[i])+str(ALL_sites[i])+'.nc is missing!')
            print(j)
            j=j+1
    #try:
    #rotated_data = rotate_stamp(ffds[i], ncfile, -9, -14)
    rotated_data = rotate_stamp(ffds[i], ncfile, meso_x[i], meso_y[i], st_2a=datetime(2020,5,26,0,45), st_2b=None)
    
    with open('SPORK_SHIFT_AC3'+str(ALL_years[i])+str(ALL_months[i])+str(ALL_days[i])+str(ALL_sites[i])+'.pkl', 'wb') as f:
        pickle.dump(rotated_data, f)
#     except:
#         print('SPORK_RERUN'+str(ALL_years[i])+str(ALL_months[i])+str(ALL_days[i])+str(ALL_sites[i])+'.nc failed!')
        

D:/SPORK_AC/SPORK_AC2020526KDFX.nc
2020-05-26 00:30:00
[9999. 9999. 1017. 1375. 1716. 2073. 2414. 2755. 3096. 3475. 3852.]
[311.0, 668.0, 1017.0, 1375.0, 1716.0, 2073.0, 2414.0, 2755.0, 3096.0, 3475.0, 3852.0]
(array([2, 3, 4, 5, 6, 7, 8, 9], dtype=int64),)
8 times


C:\Users\matts\AppData\Local\Temp\ipykernel_30940\2188689262.py:71: RuntimeWarning: Mean of empty slice
  zdr_st = np.nanmean(zdr_mn, axis=0)
C:\Users\matts\AppData\Local\Temp\ipykernel_30940\2188689262.py:72: RuntimeWarning: Mean of empty slice
  kdp_st = np.nanmean(kdp_mn, axis=0)
C:\Users\matts\AppData\Local\Temp\ipykernel_30940\2188689262.py:75: RuntimeWarning: Mean of empty slice
  zdrd_st = np.nanmean(zdrd_mn, axis=0)


0.004603962805955598
0.00424872932794557
0.003233167235348606
0.004588565727791287
0.004010639608992599
0.0038408883271229266
0.008208522947574662
0.004741849734697935
(8, 41, 140, 140)


In [18]:
print(meso_y.shape)

(409,)


In [11]:
#for i in np.arange(0,205,1):
for i in range(len(ALL_years)):

    print('SPORK_RERUN'+str(ALL_years[i])+str(ALL_months[i])+str(ALL_days[i])+str(ALL_sites[i]), i)

SPORK_RERUN201727KDGX 0
SPORK_RERUN201742KPOE 1
SPORK_RERUN2014618KFSD 2
SPORK_RERUN2015714KIND 3
SPORK_RERUN2016331KINX 4
SPORK_RERUN2016524KDDC 5
SPORK_RERUN2016827KMVX 6
SPORK_RERUN2017518KFDR 7
SPORK_RERUN2017712KMVX 8
SPORK_RERUN2018529KDDC 9
SPORK_RERUN2018611KOAX 10
SPORK_RERUN20151117KAMA 11
SPORK_RERUN201556KVNX 12
SPORK_RERUN201559KFDR 13
SPORK_RERUN2015623KDVN 14
SPORK_RERUN2015627KMVX 15
SPORK_RERUN2016510KPAH 16
SPORK_RERUN20161129KDGX 17
SPORK_RERUN201641KGWX 18
SPORK_RERUN20161130KGWX 19
SPORK_RERUN20151225KBMX 20
SPORK_RERUN201659KOAX 21
SPORK_RERUN2017328KLBB 22
SPORK_RERUN2015516KFDR 23
SPORK_RERUN2016316KILX 24
SPORK_RERUN2017228KDVN 25
SPORK_RERUN2018121KILX 26
SPORK_RERUN201933KMXX 27
SPORK_RERUN2019315KBMX 28
SPORK_RERUN201851KDDC 29
SPORK_RERUN201851KUEX 30
SPORK_RERUN201873KLNX 31
SPORK_RERUN2016718KUDX 32
SPORK_RERUN201558KLBB 33
SPORK_RERUN2015713KDVN 34
SPORK_RERUN2015527KAMA 35
SPORK_RERUN2016121KDGX 36
SPORK_RERUN2018514KSJT 37
SPORK_RERUN2015918KUEX 38
SPO

In [ ]:
plt.figure(1, figsize=(24,24))
xr2 = np.arange(0, len(rotated_data[0]), 1)
reflevs = np.arange(20, 70, 5)
zdrdlev = np.arange(1, 10, 1)
rotlev = np.arange(0.002,0.01, 0.002)
# plt.contourf(xr, xr, ref_st[4, :, :], reflevs, cmap=pyart.graph.cm_colorblind.HomeyerRainbow)
# plt.contour(xr, xr, zdrd_st[:, :]/4, zdrdlev, cmap=plt.cm.viridis, linewidths=5)
plt.contourf(rotated_data[0], rotated_data[1], rotated_data[2], reflevs, cmap=pyart.graph.cm_colorblind.HomeyerRainbow)
plt.contourf(rotated_data[0], rotated_data[1], rotated_data[7]/4, zdrdlev, cmap=plt.cm.viridis)
plt.contour(rotated_data[0], rotated_data[1], rotated_data[3], rotlev, colors='r', linewidths=4)
plt.contour(rotated_data[0], rotated_data[1], rotated_data[4], rotlev, colors='purple', linewidths=4)
plt.contour(rotated_data[0], rotated_data[1], rotated_data[5], rotlev, colors='blue', linewidths=4)

In [ ]:
print(rotated_data[2].shape)

In [ ]:
i=31
ffd_i = ffds[i]
ncfile = Dataset('D:/NewSPORK_stamps/SPORK_RERUN'+str(ALL_years[i])+str(ALL_months[i])+str(ALL_days[i])+str(ALL_sites[i])+'.nc')
print('SPORK_RERUN'+str(ALL_years[i])+str(ALL_months[i])+str(ALL_days[i])+str(ALL_sites[i])+'.nc')
ref = ncfile.variables['REFL'][:]
kdp = ncfile.variables['KDP'][:]
zdr = ncfile.variables['ZDR'][:]
cc = ncfile.variables['CC'][:]
zdr[ref < 20] = np.nan
zdr[cc < 0.9] = np.nan
kdp[ref < 20] = np.nan
kdp[cc < 0.9] = np.nan
nzdr = ncfile.variables['NZDR'][:]
nzdr[ref[:,4,:,:] < 20] = 0.0
nzdr[cc[:,4,:,:] < 0.95] = 0.0
zdrd = ncfile.variables['ZDRD'][:]
rot = ncfile.variables['ROT'][:]
times = ncfile.variables['Times'][:]
azim = ncfile.variables['azim'][:]
vel = ncfile.variables['VEL'][:]

st_1 = datetime(int(ALL_years[i]), int(ALL_months[i]), int(ALL_days[i]), int(ALL_hours[i]), 0)-timedelta(minutes=30)
print(st_1)
time_diffs = []
for j in range(len(times)):
    
    time_diff = (datetime.fromtimestamp(times[j]) - st_1).total_seconds()
    time_diffs.append(time_diff)
time_array = np.asarray(time_diffs)  
time_array[time_array<0] = 9999
print(time_array)
print(time_diffs)
print(np.where(time_array < 3600))
print(ref[time_array < 3600, :, :, :].shape[0], 'times')

ref_mn = ref[time_array < 3600, :, :, :]
zdr_mn = zdr[time_array < 3600, :, :, :]
kdp_mn = kdp[time_array < 3600, :, :, :]
cc_mn = cc[time_array < 3600, :, :, :]
rot_mn = rot[time_array < 3600, :, :, :]
zdrd_mn = zdrd[time_array < 3600, :, :]
nzdr_mn = nzdr[time_array < 3600, :, :]

ref_st = np.nanmean(ref_mn, axis=0)
zdr_st = np.nanmean(zdr_mn, axis=0)
kdp_st = np.nanmean(kdp_mn, axis=0)
cc_st = np.nanmean(cc_mn, axis=0)
rot_st = np.nanmean(rot_mn, axis=0)
zdrd_st = np.nanmean(zdrd_mn, axis=0)
nzdr_st = np.nanmean(nzdr_mn, axis=0)

nzdr_mn[np.isnan(nzdr_mn)] = 0.0

ref_pm = _run_pmm_one_variable(ref_mn)
rot_pm = _run_pmm_one_variable(rot_mn)
zdrd_pm =  _run_pmm_one_variable(zdrd_mn)
nzdr_pm =  _run_pmm_one_variable(nzdr_mn)
for q in range(zdr_mn.shape[0]):
    print(np.max(rot_mn[q,12,:,:]))
print(ref_mn.shape)

In [ ]:
print(zdr_mn.shape)
print(rot_mn.shape)

In [ ]:
plt.figure(1, figsize=(24,24))
xr = np.arange(0, ref.shape[2], 1)
reflevs = np.arange(20, 70, 5)
zdrdlev = np.arange(1, 10, 1)
rotlev = np.arange(0.002,0.01, 0.002)
# plt.contourf(xr, xr, ref_st[4, :, :], reflevs, cmap=pyart.graph.cm_colorblind.HomeyerRainbow)
# plt.contour(xr, xr, zdrd_st[:, :]/4, zdrdlev, cmap=plt.cm.viridis, linewidths=5)
plt.contourf(xr, xr, ref_pm[4, :, :], reflevs, cmap=pyart.graph.cm_colorblind.HomeyerRainbow)
plt.contour(xr, xr, zdrd_pm[:, :]/4, zdrdlev, cmap=plt.cm.viridis, linewidths=5)
plt.contour(xr, xr, rot_pm[4, :, :], rotlev, colors='r', linewidths=5)

In [ ]:
print(ffd_i)
ffd_x = 190
xr = np.arange(-70,70,1)
xra1, yra1 = DoRotation(xr, xr, RotRad=np.deg2rad((ffd_i)-180)*-1)
xri, yri, Z_interpi = interpolate_to_grid(np.ndarray.flatten(xra1), np.ndarray.flatten(yra1),
                                         np.ndarray.flatten(ref_pm[4,:,:]), hres=1,
                                         boundary_coords={'west':-50,'south':-50,'east':49,'north':49})

In [ ]:
plt.figure(1, figsize=(24,24))
#xr = np.arange(0, ref.shape[2], 1)
reflevs = np.arange(20, 70, 5)
zdrdlev = np.arange(1, 10, 1)
rotlev = np.arange(0.002,0.01, 0.002)
# plt.contourf(xr, xr, ref_st[4, :, :], reflevs, cmap=pyart.graph.cm_colorblind.HomeyerRainbow)
# plt.contour(xr, xr, zdrd_st[:, :]/4, zdrdlev, cmap=plt.cm.viridis, linewidths=5)
plt.contourf(xr, xr, ref_pm[4, :, :], reflevs, cmap=pyart.graph.cm_colorblind.HomeyerRainbow)
plt.contour(xr, xr, zdrd_pm[:, :]/4, zdrdlev, cmap=plt.cm.viridis, linewidths=5)
plt.contour(xr, xr, rot_pm[4, :, :], rotlev, colors='r', linewidths=5)
#plt.scatter(yra1,xra1)

In [ ]:
plt.figure(1, figsize=(24,24))
xr2 = np.arange(0, len(xri), 1)
reflevs = np.arange(20, 70, 5)
zdrdlev = np.arange(1, 10, 1)
rotlev = np.arange(0.002,0.01, 0.002)
# plt.contourf(xr, xr, ref_st[4, :, :], reflevs, cmap=pyart.graph.cm_colorblind.HomeyerRainbow)
# plt.contour(xr, xr, zdrd_st[:, :]/4, zdrdlev, cmap=plt.cm.viridis, linewidths=5)
plt.contourf(xri, yri, Z_interpi, reflevs, cmap=pyart.graph.cm_colorblind.HomeyerRainbow)
plt.contour(xr, xr, ref_pm[4, :, :], [40])


In [ ]:
print(ffd_i)

In [ ]:
from pyproj import Geod
import numpy as np
from datetime import datetime, timedelta

#Create geod object for later distance and area calculations
g = Geod(ellps='sphere')
def radar_motion(start_time, end_time, sloni, slati, slone, slate):

    stormdte = (np.datetime64(end_time) - np.datetime64('1970-01-01T00:00:00Z')) / np.timedelta64(1, 's')
    stormdtce = datetime.utcfromtimestamp(stormdte)
    stormdti = (np.datetime64(start_time) - np.datetime64('1970-01-01T00:00:00Z')) / np.timedelta64(1, 's')
    stormdtci = datetime.utcfromtimestamp(stormdti)

    distance_track = g.inv(sloni, slati,
                           slone, slate)
    dist_track = distance_track[2]                                        
    if distance_track[1] < 0:
        back = distance_track[1] + 360
    else:
        back = distance_track[1]
    storm_dur = stormdtce-stormdtci
    storm_sec = storm_dur.seconds
    speed = dist_track/storm_sec
    direc = back
    return speed, direc

In [ ]:
#Get observed motions for all of these cases
#Generate the file names for the postage stamps
j=1
new_speeds = []
new_directions = []

for i in range(len(ALL_years)):
#for i in np.arange(201,207,1):
#for i in [104]:
    try:
        print('D:/NewSPORK_stamps/SPORK_RERUN'+str(ALL_years[i])+str(ALL_months[i])+str(ALL_days[i])+str(ALL_sites[i])+'.nc')
        ncfile = Dataset('D:/NewSPORK_stamps/SPORK_RERUN'+str(ALL_years[i])+str(ALL_months[i])+str(ALL_days[i])+str(ALL_sites[i])+'.nc')
    except:
        try:
            ncfile = Dataset('D:/NewSPORK_stamps/SPORK_RERUN'+str(ALL_years[i])+str(ALL_months[i])+str(int(ALL_days[i])-1)+str(ALL_sites[i])+'.nc')
        except:
            print('SPORK_RERUN'+str(ALL_years[i])+str(ALL_months[i])+str(ALL_days[i])+str(ALL_sites[i])+'.nc is missing!')
            print(j)
            j=j+1
    case_loni = ncfile.variables['Lons'][:,70,70] 
    case_lati = ncfile.variables['Lats'][:,70,70] 
    case_timesi = ncfile.variables['Times'][:] 
    #print(case_loni, case_lati, case_timesi)
    
    st_1 = datetime(int(ALL_years[i]), int(ALL_months[i]), int(ALL_days[i]), int(ALL_hours[i]), 0)-timedelta(minutes=30)
    #print(st_1)
    time_diffs = []
    for j in range(len(case_timesi)):

        time_diff = (datetime.fromtimestamp(case_timesi[j]) - st_1).total_seconds()
        time_diffs.append(time_diff)
    time_array = np.asarray(time_diffs)  
    time_array[time_array<0] = 9999
    #print(time_array)
    #print(time_diffs)
    #print(np.where(time_array < 3600))
    
    case_lon = case_loni[time_array < 3600]
    case_lat = case_lati[time_array < 3600]
    case_times = case_timesi[time_array < 3600]
    
    Xlon = np.array(case_lon)
    Ylat = np.array(case_lat)
    X_obs1 = Xlon[0]
    X_obs2 = Xlon[-1]
    X_mean = np.mean(Xlon)
    Y_mean = np.mean(Ylat)
    num = 0
    den = 0
    for i in range (len(Xlon)):
        num += (Xlon[i] - X_mean)*(Ylat[i] - Y_mean)
        den += (Xlon[i] - X_mean)**2
    m = num / den
    c = Y_mean - m*X_mean
    Y_pred2 = m*X_obs2 + c
    Y_pred1 = m*X_obs1 + c
    #print("X_obs1, Y_pred1 =", X_obs1, Y_pred1, "; X_obs2, Y_pred2 =", X_obs2, Y_pred2)
    
    st_speed, st_direction = radar_motion(datetime.fromtimestamp(case_times[0]), datetime.fromtimestamp(case_times[-1]), X_obs1, Y_pred1, X_obs2, Y_pred2)
    
    print(st_speed, st_direction)
    
    new_speeds.append(st_speed)
    new_directions.append(st_direction)

In [ ]:
efilename = 'OldObsMotions.csv'
with open(efilename, mode='w') as cal_file:
    calWriter = csv.writer(cal_file, delimiter = ',', quotechar = '"', quoting = csv.QUOTE_MINIMAL)

for i in range(len(ALL_years)):

    with open(efilename, mode='a') as cal_file:
            calWriter = csv.writer(cal_file, delimiter = ',', quotechar = '"', quoting = csv.QUOTE_MINIMAL)
            calWriter.writerow([ALL_years[i], ALL_months[i], ALL_days[i], ALL_hours[i], new_speeds[i], new_directions[i]])